In [3]:
import random
import numpy as np


# Basic strategy

In [4]:
def generate_initial_candidate(size):
    board = np.zeros((size, size), dtype=int)
    numbers = list(range(1, size + 1))
    for row in range(size):
        row_numbers = numbers.copy()
        random.shuffle(row_numbers)
        board[row, :] = row_numbers
    return board

In [5]:
size = 16
initial_candidate = generate_initial_candidate(size)
print(initial_candidate)

[[ 6 15  1 12  3 10 11 14  7  2  5 16  4 13  9  8]
 [ 3  4 16 12  2  7  9  5 14  8  6  1 11 15 13 10]
 [ 3  8 14  9 10  2  4  1 13  6 12  7 16  5 15 11]
 [11  5  8  9 15  3 12  7  4 16  1  6 10 13  2 14]
 [14  9  1 10 12 15  6  7  2  5  3  4 16  8 11 13]
 [15  3  1 13  2  6  8 14 16 11  4 10  7  5 12  9]
 [11  2 13  6  3  5  9  7 10 15 16  1 12 14  4  8]
 [13  3  2 10  4  8  1 16 14  6 11 15  5  9 12  7]
 [ 1 10 13 12  7  2 15  8 14  4  6 16  3  9 11  5]
 [15  2  7 12  8  6  1 11  3 13 16  5  4 10 14  9]
 [12  4  3  6  1  5  2  8 11 15  9  7 14 10 13 16]
 [14 12  3  1  4 16  6 10  5 15 13  7  8  2 11  9]
 [11  6 16  1 10  2  7 13  3 14  5  8  9 12 15  4]
 [11  7 13  5 14  1 10 16  3  8  9 15  2 12  6  4]
 [10  6  7  5  3 13 11  8  9 16 14  2 15 12  4  1]
 [13  4 11  2  3  9 15  7  8 16 14  5  1 12 10  6]]


In [6]:
def evaluate(board): # a number of mistakes
    size = len(board)
    subgrid_size = 4
    violations = 0
    
    # Evaluate rows
    for row in board:
        violations += (size - len(set(row)))
    
    # Evaluate columns
    for col in board.T:
        violations += (size - len(set(col)))
    
    # Evaluate sub-grids
    for i in range(0, size, subgrid_size):
        for j in range(0, size, subgrid_size):
            sub_grid = board[i:i+subgrid_size, j:j+subgrid_size].flatten()
            violations += (size - len(set(sub_grid)))
    
    return violations







In [7]:
print(evaluate(initial_candidate))

173


In [8]:
def swap_random_elements(board): # change place for 2 elements in one row
    new_board = board.copy()
    row = random.randint(0, len(board) - 1)
    col1, col2 = random.sample(range(len(board)), 2)
    new_board[row, col1], new_board[row, col2] = new_board[row, col2], new_board[row, col1]
    return new_board

# Different strategies

In [9]:
def swap_random_elements_in_column(board): # change place for 2 elements in one column
    new_board = board.copy()
    col = random.randint(0, len(board) - 1)
    row1, row2 = random.sample(range(len(board)), 2)
    new_board[row1, col], new_board[row2, col] = new_board[row2, col], new_board[row1, col]
    return new_board


In [10]:
def swap_random_elements_in_subgrid(board): # change place for two elements in one subgrid
    new_board = board.copy()
    subgrid_size = 4
    subgrid_row = random.randint(0, len(board) // subgrid_size - 1) * subgrid_size
    subgrid_col = random.randint(0, len(board) // subgrid_size - 1) * subgrid_size
    row1, col1 = random.randint(0, subgrid_size - 1), random.randint(0, subgrid_size - 1)
    row2, col2 = random.randint(0, subgrid_size - 1), random.randint(0, subgrid_size - 1)
    new_board[subgrid_row + row1, subgrid_col + col1], new_board[subgrid_row + row2, subgrid_col + col2] = new_board[subgrid_row + row2, subgrid_col + col2], new_board[subgrid_row + row1, subgrid_col + col1]
    return new_board


In [11]:
def swap_random_rows(board): # swaps rows
    new_board = board.copy()
    row1, row2 = random.sample(range(len(board)), 2)
    new_board[[row1, row2], :] = new_board[[row2, row1], :]
    return new_board


In [12]:
def swap_random_columns(board): #swaps columns
    new_board = board.copy()
    col1, col2 = random.sample(range(len(board)), 2)
    new_board[:, [col1, col2]] = new_board[:, [col2, col1]]
    return new_board


#### Hill climbing

In [19]:
def hill_climbing(size, swap_function, max_restarts=10):
    best_overall_score = float('inf')
    best_overall_board = None
    
    for _ in range(max_restarts):
        current_board = generate_initial_candidate(size)
        current_score = evaluate(current_board)
        
        while True:
            best_neighbor = current_board
            best_score = current_score
            
            for _ in range(1000):
                neighbor = swap_function(current_board.copy())
                neighbor_score = evaluate(neighbor)
                
                if neighbor_score < best_score:
                    best_neighbor = neighbor
                    best_score = neighbor_score
            
            if best_score == current_score:
                break
            
            current_board = best_neighbor
            current_score = best_score
        
        if current_score < best_overall_score:
            best_overall_score = current_score
            best_overall_board = best_neighbor
    
    return best_overall_board, best_overall_score

# Initialize and run the hill climbing algorithm
size = 16
strategies = {
    'swap_random_elements': swap_random_elements,
    'swap_random_elements_in_column': swap_random_elements_in_column,
    'swap_random_elements_in_subgrid': swap_random_elements_in_subgrid,
    'swap_random_rows': swap_random_rows,
    'swap_random_columns': swap_random_columns
}

results = {}

In [20]:
for name, strategy in strategies.items():
    print(f"Testing strategy: {name}")
    final_board, final_score = hill_climbing(size, strategy)
    results[name] = final_score
    print(f"Final board for {name} strategy:")
    print(final_board)
    print(f"Score: {final_score}")

# Output the results
for strategy_name, score in results.items():
    print(f"Strategy: {strategy_name}, Score: {score}")

Testing strategy: swap_random_elements
Final board for swap_random_elements strategy:
[[ 3 11  6 16  4  7 12  5 15  8  9 10  1 13  2 14]
 [16 15  5  8 10 14 13  2  7  6  1  4  3 11  9 12]
 [ 4  9  7 10  3 16  6  1 13 11  2 14 15 12  5  8]
 [13  1  2 14 15 11  8  9 12  6  3  5 16  7  4 10]
 [12  5  9 11  1  3  6 16  2 14  4 10  8 15  7 13]
 [15 13 16  4  9  8 14 12  3  7 11  6  2  1 10  5]
 [ 8  6 10  2  7  5  4 15  9 13 16  1 12 14  3 11]
 [14  7  3  1  2 13 11 10  8 12  5 15  4 16  6  9]
 [ 9 12  4 15 14  2  3 11 16 10  7  8 13  5  1  6]
 [11  8  1  6 12 15 10  7  4  5 13  2 14  9 16  3]
 [10 16 14  5 13  1  9  4 15  3 12  6  7  8 11  2]
 [ 7  2 13  3  5 12 16  6 11  9 14  1 10  4  8 15]
 [14 10 16  9 12  4  7 13  1  2  6 11  5  3 15  8]
 [ 2  5 15 12 11  6  1  3 14 16  8  9  7 10 13  4]
 [ 1  3 11 13  8 10  2 15  5  4 12  7  9  6 14 16]
 [ 6  4  8  7 16  9  5 14 10 15  3 13 11  2 12  1]]
Score: 20
Testing strategy: swap_random_elements_in_column
Final board for swap_random_elements_i

We can see thet best resalt we have using usual random swap 